In [4]:
# 랭체인의 장점 - 각 모델 API를 알아야 할 피룡도 없고, 모델을 제공하는 기업을 알 필요도 없다. 그냥 import 해서 사용하면 된다. 다른 모든 것과 호환할 수 있다.

# from langchain.llms.openai import OpenAI
# llm = OpenAI(model_name="gpt-3.5-turbo-1106")
# a = llm.predict("How many planets are there?")

from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI()
chat.predict("How many planets are there?")




/Users/dahyeonbae/fullstack-gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/dahyeonbae/fullstack-gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('There are eight recognized planets in our solar system, which are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')